In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from dataset import BratsSupervisedDataset, BratsUnsupervisedDataset
from glob import glob
import albumentations as A

In [2]:
img_paths = glob('/mnt/asgard2/data/lingkai/braTS20/slice/image/*')
mask_paths = glob('/mnt/asgard2/data/lingkai/braTS20/slice/mask/*')

train_transform = A.Compose(
        [
        # A.CropNonEmptyMaskIfExists(height=150, width=150),
        A.HorizontalFlip(p=0.5),
        A.Affine(scale=(1.0, 1.5), p=0.15),
        A.Affine(translate_percent=(0, 0.25), p=0.5),
        A.GaussianBlur(sigma_limit=(0.5, 1.5), p=0.15), 
        A.GaussNoise(var_limit=(0, 0.33), p=0.15),
        A.RandomBrightness(limit=(0.7, 1.3), p=0.15),
        A.RandomContrast(limit=(0.65, 1.5), p=0.15)],
        additional_targets={'t1': 'image', 't1ce': 'image', 't2': 'image', 'tumorCore': 'mask', 'enhancingTumor': 'mask'}
    )
train_unsupervise_dataset = BratsUnsupervisedDataset(img_paths, augmentation=train_transform)
train_supervise_dataset = BratsSupervisedDataset(img_paths, mask_paths, augmentation=None)
train_unsupervise_loader = torch.utils.data.DataLoader(
        train_unsupervise_dataset,
        batch_size=7,
        shuffle=True,
        pin_memory=True,
        drop_last=True)

train_supervise_loader = torch.utils.data.DataLoader(
        train_supervise_dataset,
        batch_size=1,
        shuffle=True,
        pin_memory=True,
        drop_last=True)

for i, (data1, data2) in enumerate(zip(train_supervise_loader, train_unsupervise_dataset)):
    image1, target1 = data1
    image2, target2 = data2

/home/lingkai/.local/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1613: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/lingkai/.local/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


KeyboardInterrupt: 